In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#!/bin/bash
! kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.47GB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df = df.sample(n = 5000 , random_state = 42).reset_index(drop = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     5000 non-null   object
 1   sentiment  5000 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [ ]:
import re
import string
import nltk
! pip install contractions

import contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.3 MB/s eta 0:00:00


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
lemma = WordNetLemmatizer()

stop_word = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
  text = contractions.fix(text)
  text = text.lower()
  text = re.sub(r'http\S+' , '' , text)
  text = re.sub(r'<.*?>' , '',text)
  text = text.translate(str.maketrans('' , '' , string.punctuation))
  token = word_tokenize(text)
  final = [lemma.lemmatize(word) for word in token if word not in stop_word]
  return ' '.join(final)

In [ ]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [ ]:
df['review'] = df['review'].apply(clean_text)

In [ ]:
df.head()

,review,sentiment
0,really liked summerslam due look arena curtain...,1
1,many television show appeal quite many differe...,1
2,film quickly get major chase scene ever increa...,0
3,jane austen would definitely approve onegwynet...,1
4,expectation somewhat high went see movie thoug...,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(
    max_df = 0.85 ,
    min_df =1 ,
    ngram_range = (0 , 1 ),
)

In [ ]:
x = df['review']
y = df['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x, y , test_size = 0.2  , random_state = 42)

In [ ]:
x_train_vec = vector.fit_transform(x_train)

In [ ]:
x_test_vec = vector.transform(x_test)

In [ ]:
import joblib

joblib.dump(vector , 'imdb_vector.pkl')

['imdb_vector.pkl']

In [ ]:
import numpy as np

y_test = np.array(y_test)
y_train = np.array(y_train)

In [110]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [114]:
from xgboost import XGBClassifier

In [115]:
model= XGBClassifier()

In [116]:
model.fit(x_train_vec , y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [117]:
y_pred = model.predict(x_test_vec)

In [118]:
model.score(x_train_vec , y_train)

0.996

In [119]:
from sklearn.metrics import *

In [120]:
y_pred = (y_pred >= 0.5).astype(int)

In [121]:
accuracy_score(y_test, y_pred)

0.811

In [122]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       506
           1       0.80      0.83      0.81       494

    accuracy                           0.81      1000
   macro avg       0.81      0.81      0.81      1000
weighted avg       0.81      0.81      0.81      1000



In [123]:
confusion_matrix(y_test , y_pred)

array([[401, 105],
       [ 84, 410]])

In [124]:
import joblib

joblib.dump(model , 'imdb_model.pkl')

['imdb_model.pkl']